# Dewan Lab Image Analysis

## 0: Run once to create all needed directories at beginning of a project

In [ ]:
from Python.Helpers import DewanIOhandler
DewanIOhandler.createProjectFramework()

## 1: Always Execute! Load Libraries and User Settings

### 1a: Import Libraries

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import numpy as np
import pandas as pd
from tqdm import trange

from Python.Helpers import DewanDataStore
from Python.Helpers import DewanImportDataHelper
from Python.Helpers import DewanIOhandler
from Python import DewanAUROC
from Python import DewanPlotting
from Python import DewanStats
from Python import DewanManualCuration

In [15]:
DewanManualCuration.manual_curation_gui(None, None, None)

### 1b: User Configurables

In [ ]:
animal = 'VGAT4'
date = '10-17-22'

# Settings for Inscopix box
LED_power = 1
GAIN = 2.2
FOCUS = 250

PreTrialTime = 3.5  # Imaging time before the final valve opens
PostTrialTime = 3.5  # Imaging time after final valve closes

# Criterion for determining good trials vs bad trials, will be used later
goodCriterion = 0.5
# in the findGoodTrials segment


# Configurables for AUROC
baselineDuration = 2  # number of seconds before the fvOnTime
responseDuration = 2  # number of seconds after the fvOnTime
PlotFigures = True

fileHeader = animal + '-' + date + '-'


## 2: Data Import and Processing

### 2a: Import and pre-process the raw data

In [ ]:
# Read in data for processing.  Needs Cell Traces, Odor List, and GPIO file.
folder = './ImagingAnalysis/RawData/'

CellTraceData = pd.read_csv(
    f'{folder}/CellTraces.csv', header=None, skiprows=2, low_memory=False)
OdorData = np.array(pd.read_excel(
    f'{folder}/OdorList.xlsx', usecols=[0], header=None))
GPIOData = pd.read_csv(f'{folder}/GPIO.csv', header=None, low_memory=False)
CellProps = pd.read_csv(f'{folder}/CellTraces-props.csv', header=None)

In [4]:
#Manual Curation Will Happen Here

In [ ]:
# Organizes Cell Traces into a numpy array and separates the GPIO information based on indices
# for Sniff, Final Valve etc.
# Converts cell props into a numpy array, ignore the first row of headers
AllCellProps = np.array(CellProps.iloc[1:, :])
# Get indicies where status is not rejected, these are good cells
GoodCells = (AllCellProps[:, 1] != 'rejected')
# Filter the bad cells out of the properites
GoodCellProperties = AllCellProps[GoodCells, :]
# Generate a list of all the possible cell numbers and assign an index
CellList = np.arange(0, len(AllCellProps))
# Filter out all the bad cells; associates cell number with an index
CellList = CellList[GoodCells]
# Insert 1 at the beginning of the truth table, makes filtering bad cells out of trace data easier
GoodCells = np.insert(GoodCells, 0, [1])


CellTraceData = CellTraceData.apply(
    pd.to_numeric, errors='coerce')  # Convert everything to numbers
CellTraceData = np.array(CellTraceData)  # Convert pandas DF to np.array
# Filter out all the bad cell trace data, leaving only the good cells
CellTraceData = CellTraceData[:, GoodCells]
CellTraceData[:, 0] = CellTraceData[:, 0].round(2)  # Round the times to 0.2
GPIOData.iloc[:, 1] = GPIOData.iloc[:, 1].str.replace(' ', '')  # Remove Random Spaces

In [ ]:

GPIO1 = np.array(GPIOData.iloc[:, 1] == "GPIO-1")  # Sniff Sensor Data
GPIO2 = np.array(GPIOData.iloc[:, 1] == "GPIO-2")  # FV Actuation Data
# GPIO3 = np.array(GPIOData.iloc[:,1] == "GPIO-3")  # Running Wheel Data
# GPIO4 = np.array(GPIOData.iloc[:,1] == "GPIO-4")  # Lick Data

SniffTable = np.array(GPIOData.iloc[GPIO1,:])
FVData = np.array(GPIOData.iloc[GPIO2,:])

In [ ]:
# Pickle the reorganized CellTraceData incase its needed later
# Saves Cell Traces, GPIO, Odor List, Sniff, FV data, Good Cell Properties, and the list of Good Cells

# Once these have been saved, they don't need to be re-run on the same data again unless
# the data itself is changed

folder = './ImagingAnalysis/PreProcessedData'

DewanIOhandler.saveDataToDisk(
    CellTraceData, 'CellTraceData', fileHeader, folder)
DewanIOhandler.saveDataToDisk(GPIOData, 'GPIOData', fileHeader, folder)
DewanIOhandler.saveDataToDisk(OdorData, 'OdorData', fileHeader, folder)
DewanIOhandler.saveDataToDisk(SniffTable, 'SniffTable', fileHeader, folder)
DewanIOhandler.saveDataToDisk(FVData, 'FVdata', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    GoodCellProperties, 'GoodCellProperties', fileHeader, folder)
DewanIOhandler.saveDataToDisk(CellList, 'CellList', fileHeader, folder)


### 2b: Checkpoint: Load Preprocessed Data

In [ ]:
# Opens the saved pickle files.  If the files have already been saved, code can be re-run
# starting from this point
folder = 'ImagingAnalysis\PreProcessedData'

CellTraceData = DewanIOhandler.loadDataFromDisk(
    'CellTraceData', fileHeader, folder)
FVData = DewanIOhandler.loadDataFromDisk('FVData', fileHeader, folder)
GPIOData = DewanIOhandler.loadDataFromDisk('GPIOData', fileHeader, folder)
OdorData = DewanIOhandler.loadDataFromDisk('OdorData', fileHeader, folder)
SniffTable = DewanIOhandler.loadDataFromDisk('SniffTable', fileHeader, folder)
GoodCellProperties = DewanIOhandler.loadDataFromDisk(
    'GoodCellProperties', fileHeader, folder)
CellList = DewanIOhandler.loadDataFromDisk('CellList', fileHeader, folder)


### 2c: Cell Data Complilation

In [ ]:
# Parses the final valve data to identify when the final valve is open vs when it is closed.
FVValue = FVData[:, 2].astype(float)
nValue = len(FVValue)

#nOnOff = -1
valveStatus = 0
FVoffIdx = []
FVonIdx = []
for i in trange((nValue-1), desc="Processing: "):
    vDiff = FVValue[i+1] - FVValue[i]

    if valveStatus == 0:
        if vDiff > 10000:
            #nOnOff += 1
            # print(nOnOff)
            FVonIdx.append(i+1)
            valveStatus = 1
            # print(FVonIdx)
    else:
        if vDiff < -10000:
            FVoffIdx.append(i)
            valveStatus = 0

FVoffIdx = np.array(FVoffIdx)  # verified to match MTLB
FVonIdx = np.array(FVonIdx)  # verified to match MTLB


In [ ]:
# creates a list of data for the start and end time points of trials
# eliminates bad trials (trials with too short/long of a trial period etc.)
# searches through the start and ends of the good trials for matching timepoints
# and outputs indices for the on/off points of good trials
startTrialTimes = np.array(FVData[FVonIdx, 0].astype(float) - PreTrialTime)
endTrialTimes = np.array(FVData[FVoffIdx, 0].astype(float) + PostTrialTime)
CellTraceonIdx, goodTrialsOn = DewanImportDataHelper.findGoodTrials(CellTraceData, startTrialTimes, goodCriterion)
CellTraceoffIdx, goodTrialsOff = DewanImportDataHelper.findGoodTrials(CellTraceData, endTrialTimes, goodCriterion)
goodTrials, goodIndOn, goodIndOff = DewanImportDataHelper.intersect_matlab(goodTrialsOn, goodTrialsOff)


In [ ]:
# Index all data by the good trials to exclude data from bad trials

CellTraceonIdx = CellTraceonIdx[goodIndOn.astype(int)]
CellTraceoffIdx = CellTraceoffIdx[goodIndOff.astype(int)]

# https://knowyourmeme.com/photos/1402268 ?????
goodIdx = goodTrials.astype(int)-1

FVonIdx = FVonIdx[goodTrials.astype(int)]
FVoffIdx = FVoffIdx[goodTrials.astype(int)]
startTrialTimes = startTrialTimes[goodTrials.astype(int)]
endTrialTimes = endTrialTimes[goodTrials.astype(int)]
OdorData = OdorData[goodTrials.astype(int)]


#### Sniffing Data Compilation

In [ ]:
# Compiles data for sniffing from good trialss
sniffStartIdx = np.zeros(FVonIdx.size)
sniffEndIdx = np.zeros(FVonIdx.size)

for i in trange(len(goodTrials), desc="Trial: "):
    startTrial_Time_s_ = float(FVData[:, 0][FVonIdx[i]]) - PreTrialTime
    endTrial_Time_s_ = float(FVData[:, 0][FVoffIdx[i]]) + PostTrialTime
    sniffStartIdx[i] = len(np.nonzero(
        SniffTable[:, 0].astype(float) < startTrial_Time_s_)[0])
    sniffEndIdx[i] = len(np.nonzero(
        SniffTable[:, 0].astype(float) < endTrial_Time_s_)[0]) - 1

nSniffPoints = np.max(sniffEndIdx - sniffStartIdx)

Sniff = np.zeros((nSniffPoints.astype(int), len(sniffStartIdx)))

for numi in trange(len(sniffStartIdx), desc="Trial: "):
    nSniff = sniffEndIdx[numi] - sniffStartIdx[numi]
    Sniff[0:int(nSniff), numi] = SniffTable[sniffStartIdx[numi].astype(
        int):sniffEndIdx[numi].astype(int), 2]

DewanIOhandler.saveDataToDisk(
    Sniff, 'SniffData', fileHeader, 'PreProcessedData')


### 2d: Gather all **cell v. time v. trial** data into single array

In [ ]:
nDataPoint = np.max(CellTraceoffIdx - CellTraceonIdx)
Data = np.zeros((len(CellList), len(CellTraceoffIdx), nDataPoint.astype(int)))

for cells in range(len(CellList)):
    for trials in range(len(CellTraceonIdx)):
        nDataPoints = (CellTraceoffIdx[trials] - CellTraceonIdx[trials])
        Data[cells, trials, :nDataPoints[0].astype(int)] = CellTraceData[CellTraceonIdx[trials][0].astype(
            int):CellTraceoffIdx[trials][0].astype(int), cells+1]

Data = np.delete(Data, 180, 2)
Baseline_Shift_Data = Data + abs(np.min(Data))


In [ ]:
# Map of trial times relative to startTrial time
FVTimeMap = np.zeros((len(CellTraceoffIdx), nDataPoint.astype(int)))
# Map of times per trial in unix time for AUROC
UnixTimeArray = np.zeros((len(CellTraceoffIdx), max(
    CellTraceoffIdx - CellTraceonIdx).astype(int)[0]))

for trial in range(len(CellTraceoffIdx)):
    nDataPoints = (CellTraceoffIdx[trial] - CellTraceonIdx[trial])
    FVTimeMap[trial, :nDataPoints[0].astype(int)] = CellTraceData[CellTraceonIdx[trial][0].astype(
        int):CellTraceoffIdx[trial][0].astype(int), 0] - float(FVData[FVonIdx[trial], 0])
    UnixTimeArray[trial, :nDataPoints[0].astype(int)] = CellTraceData[CellTraceonIdx[trial][0].astype(
        int):CellTraceoffIdx[trial][0].astype(int), 0]

FVTimeMap = np.delete(FVTimeMap, 180, 1)
UnixTimeArray = np.delete(UnixTimeArray, 180, 1)


### 2e: Combined Data Output

In [ ]:
odorIndexes = []

for i in OdorData:
    odorIndexes.append(i[0])

columnNames = ['CentroidX', 'CentroidY', 'NumComponents', 'Size']
TOC = pd.DataFrame(GoodCellProperties[:, 5:9], CellList, columnNames)

path = f'./ImagingAnalysis/CombinedData/{fileHeader}CombinedData.xlsx'
writer = pd.ExcelWriter(path, engine='xlsxwriter')

TOC.to_excel(writer, sheet_name='TOC')
TimeMapSheet = pd.DataFrame(
    FVTimeMap[:, :180], np.arange(1, len(CellTraceoffIdx)+1))
TimeMapSheet.to_excel(writer, sheet_name='TimeMap')

for i in trange(len(CellList), desc="Writing Cell: "):
    DataDf = pd.DataFrame(Baseline_Shift_Data[i][:][:], odorIndexes)
    #DataDf.index = indexes
    DataDf.to_excel(writer, sheet_name='Cell %d' % CellList[i])

writer.close()

#headers = []

# for i in range(len(Sniff[0,:])):
# headers.append('Trial ' + str(i))

#SniffDF = pd.DataFrame(Sniff)
#SniffDF.to_excel('./ImportOutputs/' + fileHeader + 'SniffData.xlsx', sheet_name = 'Data', header=headers)


### 2f: Save information needed for AUROC

In [ ]:
DewanIOhandler.saveDataToDisk(Data, 'CombinedData', fileHeader, './ImagingAnalysis/CombinedData')

folder = './ImagingAnalysis/AUROCImports'
DewanIOhandler.saveDataToDisk(FVonIdx, 'FVonIdx', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    UnixTimeArray, 'UnixTimeArray', fileHeader, folder)
DewanIOhandler.saveDataToDisk(FVTimeMap, 'FVTimeMap', fileHeader, folder)
DewanIOhandler.saveDataToDisk(OdorData, 'OdorData', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    Baseline_Shift_Data, 'Baseline_Shift_Data', fileHeader, folder)


## 3: AUROC

### 3a: Import Data 

In [ ]:
Data = DewanIOhandler.loadDataFromDisk('CombinedData', fileHeader, './ImagingAnalysis/CombinedData')

folder = 'ImagingAnalysis/PreProcessedData'
FVData = DewanIOhandler.loadDataFromDisk('FVData', fileHeader, folder)
CellList = DewanIOhandler.loadDataFromDisk('CellList', fileHeader, folder)

folder = 'ImagingAnalysis/AUROCImports'
FVonIdx = DewanIOhandler.loadDataFromDisk('FVonIdx', fileHeader, folder)
UnixTimeArray = DewanIOhandler.loadDataFromDisk(
    'UnixTimeArray', fileHeader, folder)
OdorData = DewanIOhandler.loadDataFromDisk('OdorData', fileHeader, folder)
FVTimeMap = DewanIOhandler.loadDataFromDisk('FVTimeMap', fileHeader, folder)
Baseline_Shift_Data = DewanIOhandler.loadDataFromDisk(
    'Baseline_Shift_Data', fileHeader, folder)


### 3b: Run AUROC

In [ ]:
AUROCData = DewanDataStore.AUROCdataStore(
    Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FVonIdx, UnixTimeArray, baselineDuration, responseDuration, PlotFigures)
onTimeReturnValues = DewanAUROC.AUROC(AUROCData, False)
latentReturnValues = DewanAUROC.AUROC(AUROCData, True)

In [ ]:
onTimeAUROCSignificanceTable = []
onTimeAUROCValueTable = []
onTimeAUROC_Ubounds = []
onTimeAUROC_Lbounds = []
onTimeAUROC_prctile = []


latentAUROCSignificanceTable = []
latentAUROCValueTable = []
latentAUROC_Ubounds = []
latentAUROC_Lbounds = []
latentAUROC_prctile = []

baseline_start_indexes = []
baseline_end_indexes = []
ontime_evoked_start_indexes = []
ontime_evoked_end_indexes = []
latent_evoked_start_indexes = []
latent_evoked_end_indexes = []

for returns in onTimeReturnValues:
    onTimeAUROCSignificanceTable.append(returns.response_chart)
    onTimeAUROCValueTable.append(returns.auroc_values)
    onTimeAUROC_Ubounds.append(returns.all_upper_bounds)
    onTimeAUROC_Lbounds.append(returns.all_lower_bounds)
    onTimeAUROC_prctile.append(returns.percentiles)
    baseline_start_indexes.append(returns.baseline_start_indexes)
    baseline_end_indexes.append(returns.baseline_end_indexes)
    ontime_evoked_start_indexes.append(returns.evoked_start_indexes)
    ontime_evoked_end_indexes.append(returns.evoked_end_indexes)


for returns in latentReturnValues:
    latentAUROCSignificanceTable.append(returns.response_chart)
    latentAUROCValueTable.append(returns.auroc_values)
    latentAUROC_Ubounds.append(returns.all_upper_bounds)
    latentAUROC_Lbounds.append(returns.all_lower_bounds)
    latentAUROC_prctile.append(returns.percentiles)
    latent_evoked_start_indexes.append(returns.evoked_start_indexes)
    latent_evoked_end_indexes.append(returns.evoked_end_indexes)

onTimeAUROCSignificanceTable = np.array(onTimeAUROCSignificanceTable)
latentAUROCSignificanceTable = np.array(latentAUROCSignificanceTable)
onTimeAUROCSignificanceTable = onTimeAUROCSignificanceTable.astype(float)
latentAUROCSignificanceTable = latentAUROCSignificanceTable.astype(float)

#baseline_start_indexes = np.array(baseline_start_indexes)
# baseline_end_indexes = np.array(baseline_end_indexes)
# ontime_evoked_start_indexes = np.array(ontime_evoked_start_indexes)
# ontime_evoked_end_indexes = np.array(ontime_evoked_end_indexes)
# latent_evoked_start_indexes = np.array(latent_evoked_start_indexes)
# latent_evoked_end_indexes = np.array(latent_evoked_end_indexes)

for rows in range(len(latentAUROCSignificanceTable)):
    evokedCells = np.nonzero(onTimeAUROCSignificanceTable[rows] != 0)
    latentAUROCSignificanceTable[rows][evokedCells] = 0

### 3c: Save AROC

In [ ]:
folder = 'ImagingAnalysis/AUROCData'

DewanIOhandler.saveDataToDisk(
    onTimeAUROCSignificanceTable, 'onTimeAUROCSignificanceTable', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    onTimeAUROCValueTable, 'onTimeAUROCValueTable', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    onTimeAUROC_Lbounds, 'onTimeAUROC_Lbounds', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    onTimeAUROC_Ubounds, 'onTimeAUROC_Ubounds', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    onTimeAUROC_prctile, 'onTimeAUROC_prctile', fileHeader, folder)

DewanIOhandler.saveDataToDisk(
    baseline_start_indexes, 'baseline_start_indexes', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    baseline_end_indexes, 'baseline_end_indexes', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    ontime_evoked_start_indexes, 'ontime_evoked_start_indexes', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    ontime_evoked_end_indexes, 'ontime_evoked_end_indexes', fileHeader, folder)

DewanIOhandler.saveDataToDisk(
    latentAUROCSignificanceTable, 'latentAUROCSignificanceTable', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    latentAUROCValueTable, 'latentAUROCValueTable', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    latentAUROC_Lbounds, 'latentAUROC_Lbounds', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    latentAUROC_Ubounds, 'latentAUROC_Ubounds', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    latentAUROC_prctile, 'latentAUROC_prctile', fileHeader, folder)

DewanIOhandler.saveDataToDisk(
    latent_evoked_start_indexes, 'latent_evoked_start_indexes', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    latent_evoked_end_indexes, 'latent_evoked_end_indexes', fileHeader, folder)

## 4: Plotting

### 4a: Import Data

In [ ]:
folder = 'ImagingAnalysis/AUROCData'

onTimeAUROCSignificanceTable = DewanIOhandler.loadDataFromDisk(
    'onTimeAUROCSignificanceTable', fileHeader, folder)
onTimeAUROCValueTable = DewanIOhandler.loadDataFromDisk(
    'onTimeAUROCValueTable', fileHeader, folder)
onTimeAUROC_Lbounds = DewanIOhandler.loadDataFromDisk(
    'onTimeAUROC_Lbounds', fileHeader, folder)
onTimeAUROC_Ubounds = DewanIOhandler.loadDataFromDisk(
    'onTimeAUROC_Ubounds', fileHeader, folder)
onTimeAUROC_prctile = DewanIOhandler.loadDataFromDisk(
    'onTimeAUROC_prctile', fileHeader, folder)

latentAUROCSignificanceTable = DewanIOhandler.loadDataFromDisk(
    'latentAUROCSignificanceTable', fileHeader, folder)
latentAUROCValueTable = DewanIOhandler.loadDataFromDisk(
    'latentAUROCValueTable', fileHeader, folder)
latentAUROC_Lbounds = DewanIOhandler.loadDataFromDisk(
    'latentAUROC_Lbounds', fileHeader, folder)
latentAUROC_Ubounds = DewanIOhandler.loadDataFromDisk(
    'latentAUROC_Ubounds', fileHeader, folder)
latentAUROC_prctile = DewanIOhandler.loadDataFromDisk(
    'latentAUROC_prctile', fileHeader, folder)


baseline_start_indexes = DewanIOhandler.loadDataFromDisk(
    'baseline_start_indexes', fileHeader, folder)
baseline_end_indexes = DewanIOhandler.loadDataFromDisk(
    'baseline_end_indexes', fileHeader, folder)
ontime_evoked_start_indexes = DewanIOhandler.loadDataFromDisk(
    'ontime_evoked_start_indexes', fileHeader, folder)
ontime_evoked_end_indexes = DewanIOhandler.loadDataFromDisk(
    'ontime_evoked_end_indexes', fileHeader, folder)
latent_evoked_start_indexes = DewanIOhandler.loadDataFromDisk(
    'latent_evoked_start_indexes', fileHeader, folder)
latent_evoked_end_indexes = DewanIOhandler.loadDataFromDisk(
    'latent_evoked_end_indexes', fileHeader, folder)

Data = DewanIOhandler.loadDataFromDisk(
    'CombinedData', fileHeader, 'ImagingAnalysis/CombinedData')

folder = 'ImagingAnalysis/PreProcessedData'
FVData = DewanIOhandler.loadDataFromDisk('FVData', fileHeader, folder)
CellList = DewanIOhandler.loadDataFromDisk('CellList', fileHeader, folder)

folder = 'ImagingAnalysis/AUROCImports'
FVonIdx = DewanIOhandler.loadDataFromDisk('FVonIdx', fileHeader, folder)
UnixTimeArray = DewanIOhandler.loadDataFromDisk(
    'UnixTimeArray', fileHeader, folder)
OdorData = DewanIOhandler.loadDataFromDisk('OdorData', fileHeader, folder)
FVTimeMap = DewanIOhandler.loadDataFromDisk('FVTimeMap', fileHeader, folder)
Baseline_Shift_Data = DewanIOhandler.loadDataFromDisk(
    'Baseline_Shift_Data', fileHeader, folder)


### 4b: Cells v. Odors AUROC Significance

In [ ]:
onTimePlottingData = DewanDataStore.PlottingDataStore(Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FVTimeMap, FVonIdx, UnixTimeArray, baselineDuration, responseDuration, onTimeAUROCSignificanceTable, onTimeAUROC_Lbounds, onTimeAUROC_Ubounds, onTimeAUROC_prctile, baseline_start_indexes, baseline_end_indexes, ontime_evoked_start_indexes, ontime_evoked_end_indexes)

latentPlottingData = DewanDataStore.PlottingDataStore(Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FVTimeMap, FVonIdx, UnixTimeArray, baselineDuration, responseDuration, latentAUROCSignificanceTable, latentAUROC_Lbounds, latentAUROC_Ubounds, latentAUROC_prctile, baseline_start_indexes, baseline_end_indexes, latent_evoked_start_indexes, latent_evoked_end_indexes)

#### 4b-1: Plot Cell v Odor Significance Matricies

In [ ]:
DewanPlotting.plotCellvOdorMatricies(onTimePlottingData, False)
DewanPlotting.plotCellvOdorMatricies(latentPlottingData, True)

#### 4b-2: Plot All Cell Traces

In [ ]:
# Plot all cells v odors; only needed for AUROC verification
DewanPlotting.plotAllCells(onTimePlottingData, False)
DewanPlotting.plotAllCells(latentPlottingData, True)

#### 4b-3: Plot Significant Cell Traces

In [ ]:
# Oh no! Nothing here yet :(
# https://media.tenor.com/vMvR0gEolCEAAAAC/nothing-to-see-here-explosion.gif

### 4c: Plot Trace Variance

In [ ]:
trialScatterData = DewanDataStore.AUROCdataStore(
    Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FVonIdx, UnixTimeArray, baselineDuration, responseDuration, False)


DewanPlotting.plotTrialsPerPairing(
    trialScatterData, onTimeAUROCSignificanceTable, False)
DewanPlotting.plotTrialsPerPairing(
    trialScatterData, onTimeAUROCSignificanceTable, True)


## 5. Statistics

### 5a. Load Data

In [ ]:
#Oh no, nothing here yet :(

### 5b. Correlation Coefficients

In [ ]:
similarityData = DewanDataStore.AUROCdataStore(Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FVonIdx, UnixTimeArray, baselineDuration, responseDuration, False)
CCs = DewanStats.crossTrialConsistency(similarityData, onTimeAUROCSignificanceTable, False)

In [ ]:
mineralOilIndex = np.nonzero(similarityData.unique_odors == 'MO')[0]
responsiveCellListIndex = np.nonzero(onTimeAUROCSignificanceTable[:, mineralOilIndex] == 0)[0]
odorIndexes = np.nonzero(similarityData.unique_odors != 'MO')[0]
columns = [f'Cell: {similarityData.Cell_List[cell]}' for cell in responsiveCellListIndex]
rows = [f'{similarityData.unique_odors[odor]}' for odor in odorIndexes]
CCs = np.array(CCs)
CCs = pd.DataFrame(CCs.transpose(), columns=columns, index=rows)

### 5c. Lifetime and Population Sparseness

In [ ]:
sparsenessData = DewanDataStore.AUROCdataStore(
    Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FVonIdx, UnixTimeArray, baselineDuration, responseDuration, False)

lifetimeSparseness, cells = DewanStats.lifetimeSparseness(sparsenessData, onTimeAUROCSignificanceTable)
populationSparseness, odors = DewanStats.popSparseness(sparsenessData, onTimeAUROCSignificanceTable)
lsPairs = [f'Cell {cells[i]}: {round(lifetimeSparseness[i],5)}' for i in range(len(cells))]
psPairs = [f'Odor: {sparsenessData.unique_odors[odors[i]]}: {round(populationSparseness[i],5)}' for i in range(len(odors))]

In [ ]:
mineralOilIndex = np.nonzero(sparsenessData.unique_odors == 'MO')[0]
odorIndexes = np.nonzero(sparsenessData.unique_odors != 'MO')[0]
